In [28]:
import importlib

import plotly.graph_objects as go

import apollo.providers.price_data_provider as pdp
from apollo.settings import (
    FREQUENCY,
    TICKER,
    VIX_TICKER,
)
from apollo.utils.common import to_default_date_string

importlib.reload(pdp);

In [29]:
# ruff: noqa

start_date = "2023-01-01"
end_date = "2023-07-01"

vix_price_data_provider = pdp.PriceDataProvider(
    ticker=str(VIX_TICKER),
    frequency=str(FREQUENCY),
    # start_date=str(START_DATE),
    # end_date=str(END_DATE),
    # max_period=bool(MAX_PERIOD),
    start_date=start_date,
    end_date=end_date,
    max_period=False,
)

vix_price_data = vix_price_data_provider.get_price_data()

price_data_provider = pdp.PriceDataProvider(
    ticker=str(TICKER),
    frequency=str(FREQUENCY),
    # start_date=str(START_DATE),
    # end_date=str(END_DATE),
    # max_period=bool(MAX_PERIOD),
    start_date=start_date,
    end_date=end_date,
    max_period=False,
)

price_data = price_data_provider.get_price_data()

price_data["vix close"] = vix_price_data["close"]
price_data["vix_range"] = abs(vix_price_data["high"] - vix_price_data["low"])

In [30]:
prime_value = "close"
trace_value = "vix_range"

x = price_data.index.to_numpy()

y1 = price_data[prime_value].to_numpy()
y2 = price_data[trace_value].to_numpy()

# Create the first trace with the primary y-axis
trace1 = go.Scatter(x=x, y=y1, name=prime_value)

# Create the second trace with the secondary y-axis
trace2 = go.Scatter(x=x, y=y2, name=f"{trace_value}", yaxis="y2")

# Plot title
title = (
    f"{price_data.iloc[0]["ticker"]} | VIX"
    f" {to_default_date_string(price_data.index.to_numpy()[0])}"
    f" - {to_default_date_string(price_data.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)